In [1]:
headers={
"Accept": "application/json, text/plain, */*",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9",
# "Cache-Control": no-cache
"Connection": "keep-alive",
"Content-Length": "197",
"Content-Type": "application/json",
"Host": "qutrub.arabeyes.org",
"Origin": "https://qutrub.arabeyes.org",
"Pragma": "no-cache",
# "Referer": "https://qutrub.arabeyes.org/?verb=%D8%B3%D9%85%D8%AD",
"Sec-Fetch-Dest": "empty",
"Sec-Fetch-Mode": "cors",
"Sec-Fetch-Site": "same-origin",
"Sec-GPC": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
}

req_body =  '{"data": {"text":"لعب", "action":"Conjugate","all":true,"transitive":true,"past":true,"future":true,"imperative":true,"future_moode":true,"confirmed":true,"passive":true,"future_type":"فتحة"}}'



In [2]:
import requests
from bs4 import BeautifulSoup
from itertools import zip_longest

E:\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import preprocess

In [26]:
# load verbs to fetch their conjugs later
def load_total_verbs():
    lines = []
    total_verbs = []

    with open("../../Data/verb_dictionary.txt", "r", encoding="utf-8") as file:
        lines = file.readlines()

    total_verbs = lines[0].split(" ")
    return total_verbs

In [25]:
# url = "https://qutrub.arabeyes.org/ajaxGet",
verb = str(input()),
req_body =  '{"data": {"text":"%s", "action":"Conjugate","all":true,"transitive":true,"past":true,"future":true,"imperative":true,"future_moode":true,"confirmed":true,"passive":true,"future_type":"فتحة"}}'%verb

res = requests.post(url= "https://qutrub.arabeyes.org/ajaxGet",  data= req_body.encode("utf-8"), headers=headers)
data = res.json()["result"]

In [99]:
def get_set_of_processed_conjugs(fetched_data):
    # [0] : first row (نوع التصريف)
    # first col in each other row is the pronoun, other cols are conjugations (تصريفات)
    # use from data['1']
    try:
        set_of_conjugs = set([])
        for key in fetched_data:
            if key != "0":
                for key2 in fetched_data[key]:
                    processed_word = preprocess.pre_process(fetched_data[key][key2])
                    if len(processed_word) and key2 != "0":
                        # if key2 = 0 => pronoun
                        set_of_conjugs.add(processed_word)
    except:
        #TODO there are some suggestions about the verb not a sure result, make a decision
        pass

    return set_of_conjugs

def fetch_and_get_conjugs(verb):
    req_body =  '{"data": {"text":"%s", "action":"Conjugate","all":true,"transitive":true,"past":true,"future":true,"imperative":true,"future_moode":true,"confirmed":true,"passive":true,"future_type":"فتحة"}}'%verb
    res = requests.post(url= "https://qutrub.arabeyes.org/ajaxGet",  data= req_body.encode("utf-8"), headers=headers)
    result = res.json()["result"]
    sugg = res.json()["suggest"]

    if len(result) == 0 and len(sugg) != 0:
        #TODO, try to make a better decision about the choice
        return fetch_and_get_conjugs(sugg[0]["verb"])
    elif len(result) == 0 and len(sugg) == 0:
        # no result, no suggestions
        return set()
    
    # found a result
    set_of_conjugs = get_set_of_processed_conjugs(result)
    return set_of_conjugs

def get_valid_lemm_data(l, r, verbs, lemm_data, lone_verbs):
    # return list of tuples => [ (verb , [conjugs]) ]

    for verb in verbs[l:r]:
        l = list(fetch_and_get_conjugs(verb))
        if len(l) != 0:
            lemm_data.append((verb, l))
        else:
            lone_verbs.append(verb)


In [100]:
lemm_data = []
lone_verbs = []

# load verbs to fetch their conjugs later
verbs = load_total_verbs()

In [101]:
get_valid_lemm_data(0, 100, verbs, lemm_data, lone_verbs)

In [103]:
get_valid_lemm_data(100, 1000, verbs, lemm_data, lone_verbs)

In [ ]:
get_valid_lemm_data(1000, 2000, verbs, lemm_data, lone_verbs)

In [ ]:
get_valid_lemm_data(2000, 3000, verbs, lemm_data, lone_verbs)

In [ ]:
get_valid_lemm_data(3000, 4000, verbs, lemm_data, lone_verbs)

In [ ]:
get_valid_lemm_data(4000, 5000, verbs, lemm_data, lone_verbs)

In [ ]:
get_valid_lemm_data(5000, 6000, verbs, lemm_data, lone_verbs)

In [ ]:
get_valid_lemm_data(6000, 7000, verbs, lemm_data, lone_verbs)

In [ ]:
get_valid_lemm_data(7000, len(verbs), verbs, lemm_data, lone_verbs)

980

In [146]:
with open("../../Data/Lemmatization/lemm_data.txt", "w", encoding="utf-8") as f:
    for item in lemm_data:
        f.write( "%s=>%s\n"%(item[0], " ".join(item[1])))


In [168]:
# load conjugs

# data = []
# with open("../../Data/Lemmatization/lemm_data.txt", "r", encoding="utf-8") as f:
#     data = f.read()
#     data = data.split("\n")
#     data.pop()
#     data = [line.split("=>") for line in data]
#     data = [(line[0], line[1].split(" ")) for line in data]

# data

[('امض',
  ['امضتا',
   'اءمضنان',
   'امضتن',
   'امضت',
   'اءمضان',
   'تامضا',
   'اءمضن',
   'يامض',
   'امضتم',
   'اءمض',
   'نامض',
   'يامضوا',
   'يامضا',
   'تامضنان',
   'يامضن',
   'امض',
   'تامضن',
   'امضا',
   'اءمضوا',
   'امضنا',
   'تامض',
   'يامضنان',
   'امضن',
   'يامضان',
   'تامضان',
   'تامضون',
   'يامضون',
   'امضتما',
   'اءمضا',
   'نامضن',
   'اءمضي',
   'تامضين',
   'تامضوا',
   'امضوا',
   'تامضي']),
 ('صحف',
  ['صحفتا',
   'يصحفان',
   'يصحفن',
   'صحفوا',
   'نصحف',
   'يصحف',
   'صحفا',
   'صحف',
   'تصحفين',
   'صحفنا',
   'صحفتما',
   'صحفتن',
   'تصحفنان',
   'صحفن',
   'اصحف',
   'تصحفا',
   'اصحفوا',
   'تصحفان',
   'يصحفا',
   'اصحفنان',
   'تصحفن',
   'اصحفا',
   'تصحفون',
   'نصحفن',
   'تصحفي',
   'تصحف',
   'يصحفوا',
   'تصحفوا',
   'يصحفنان',
   'اصحفي',
   'اصحفان',
   'اصحفن',
   'صحفت',
   'صحفتم',
   'يصحفون']),
 ('حجو',
  ['تحجوان',
   'يحجي',
   'حجونا',
   'حجون',
   'حجي',
   'احجون',
   'احج',
   'حجوا',
   'احجوا',
   'حجوتا',
 

In [ ]:
s = "abds"
s.